### Conversational AI (Chatbot)

In [25]:
import os
from dotenv import load_dotenv
import gradio as gr
import google.generativeai as genai
from IPython.display import Markdown, display, clear_output

In [26]:
load_dotenv()

google_api_key = os.getenv('GEMINI_API_KEY')

In [27]:
genai.configure(api_key=google_api_key)

In [28]:
system_message = "You are a helpful assistant"

In [29]:
def message_gemini(prompt):
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",  # or "gemini-1.5-pro"
        system_instruction=system_message
    )
    
    response = model.generate_content(prompt)
    return response.text


In [30]:
view = gr.Interface(
    fn=message_gemini,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


### Streaming Output

In [31]:

system_message = "You are a helpful assistant that responds in markdown"  # Optional system instruction

def stream_gemini(prompt):
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",
        system_instruction=system_message
    )

    stream = model.generate_content(prompt, stream=True)

    result = ""
    for chunk in stream:
        if chunk.text:
            result += chunk.text
            yield result


In [32]:
for partial in stream_gemini("Explain quantum computing in simple terms."):
    clear_output(wait=True)
    display(Markdown(partial))

Okay, imagine regular computers (the ones we use every day) like light switches. Each switch can be either on (1) or off (0). These are called bits, and they're the fundamental building blocks of how computers store and process information.

Quantum computers, on the other hand, are like dimmer switches. They can be on (1), off (0), **or** somewhere in between (both 1 and 0 *at the same time*).  This "somewhere in between" state is called **superposition**.

Here's a breakdown of the key ideas:

*   **Qubits:** Instead of bits, quantum computers use **qubits**.  A qubit can be 0, 1, or a *superposition* of both 0 and 1. Think of it like a spinning coin - it's neither heads nor tails until it lands.

*   **Superposition:** This allows a qubit to represent multiple possibilities simultaneously.  Instead of exploring one possibility at a time (like a regular computer), a quantum computer can explore many possibilities at once.  This gives them incredible power for certain kinds of problems.

*   **Entanglement:**  This is where things get really weird. Imagine two of our spinning coins that are linked together in a special way. If you look at one and it lands on heads, you instantly know the other one is tails, even if they're miles apart.  That's kind of like entanglement. Entangled qubits are linked, so the state of one affects the state of the other, no matter the distance. This allows qubits to work together and solve problems in ways classical computers can't.

*   **Measurement:** When you finally need an answer from a quantum computer, you "measure" the qubits.  This forces them to "choose" a state (either 0 or 1), and you get the result. However, the act of measuring destroys the superposition and entanglement.

**Why is this useful?**

Quantum computers are not going to replace your laptop anytime soon. They're really good at solving specific types of problems that are incredibly difficult for regular computers, such as:

*   **Drug Discovery:** Simulating molecules to design new medicines.
*   **Materials Science:** Discovering new materials with specific properties.
*   **Cryptography:** Breaking existing encryption and creating new, more secure methods.
*   **Optimization:** Finding the best solution to complex problems, like optimizing delivery routes or financial portfolios.
*   **Artificial Intelligence:** Training AI models faster and more efficiently.

**In summary:**

Quantum computers use qubits that can be in a superposition of 0 and 1, and they can be entangled with each other.  This allows them to explore many possibilities at once and solve certain complex problems much faster than classical computers.  They are not a replacement for your current computer, but a specialized tool for tackling specific scientific and technological challenges.


In [33]:
view = gr.Interface(
    fn=stream_gemini,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=gr.Markdown(),
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.


### Chat Bot With History

In [34]:
def stream_gemini_with_history(message, history):
        
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",
        system_instruction=system_message
    )
    
    prompt = ""
    for user_msg, bot_msg in history:
        prompt += f"User: {user_msg}\nAssistant: {bot_msg}\n"
    prompt += f"User: {message}\nAssistant:"

    stream = model.generate_content(prompt, stream=True)

    response = ""
    for chunk in stream:
        if chunk.text:
            response += chunk.text
            yield history + [(message, response)], ""


In [35]:
def clear_chat():
    return [], ""

with gr.Blocks(theme=gr.themes.Soft()) as view:
    gr.Markdown("## 💬 Chat Bot with History")

    chatbot = gr.Chatbot(label="Conversation", show_copy_button=True)
    msg = gr.Textbox(label="Your message:", lines=2, placeholder="Type your message here...")

    with gr.Row():
        send_btn = gr.Button("Send")
        clear_btn = gr.Button("Clear Chat")

    msg.submit(stream_gemini_with_history, inputs=[msg, chatbot], outputs=[chatbot, msg])
    send_btn.click(stream_gemini_with_history, inputs=[msg, chatbot], outputs=[chatbot, msg])

    clear_btn.click(clear_chat, outputs=[chatbot, msg])

view.launch()


C:\Users\chandula\AppData\Local\Temp\ipykernel_15184\1344445376.py:7: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation", show_copy_button=True)


* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.
